<a href="https://colab.research.google.com/github/pjd-code/millipede-microplastic/blob/main/millipede_microplastic_qiime%2Bgreengenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will quickly run through the steps I used to process the raw reads. 

In [1]:
#clean up the Colab environment
%cd /content/sample_data
!rm *.csv
!rm *.md
!rm *.json
%cd /content/sample_data

/content/sample_data
/content/sample_data


In [ ]:
from google.colab import drive
drive.mount('/content/sample_data/googledrive')

In [ ]:
!git clone https://github.com/pjd-code/millipede-microplastic.git

In [ ]:
%run /content/sample_data/hudson-valley-tick/setup_qiime2.py

In [4]:
%cd /bin
!wget "https://launch.basespace.illumina.com/CLI/latest/amd64-linux/bs" -O /bin/bs
!chmod u+x /bin/bs

/bin
--2022-04-22 13:28:54--  https://launch.basespace.illumina.com/CLI/latest/amd64-linux/bs
Resolving launch.basespace.illumina.com (launch.basespace.illumina.com)... 13.225.71.115, 13.225.71.76, 13.225.71.58, ...
Connecting to launch.basespace.illumina.com (launch.basespace.illumina.com)|13.225.71.115|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12218368 (12M) [binary/octet-stream]
Saving to: ‘/bin/bs’

/bin/bs             100%[===================>]  11.65M  --.-KB/s    in 0.1s    

2022-04-22 13:28:54 (83.0 MB/s) - ‘/bin/bs’ saved [12218368/12218368]



In [5]:
!bs auth
!bs  whoami

Please go to this URL to authenticate:  https://basespace.illumina.com/oauth/device?code=hMFHJ
Welcome, Preston Dihle
+----------------+----------------------------------------------------+
| Name           | Preston Dihle                                      |
| Id             | 23726704                                           |
| Email          | preston.dihle@gmail.com                            |
| DateCreated    | 2021-02-05 02:03:22 +0000 UTC                      |
| DateLastActive | 2022-04-22 13:29:12 +0000 UTC                      |
| Host           | https://api.basespace.illumina.com                 |
| Scopes         | READ GLOBAL, CREATE GLOBAL, BROWSE GLOBAL,         |
|                | CREATE PROJECTS, CREATE RUNS, START APPLICATIONS,  |
|                | MOVETOTRASH GLOBAL, WRITE GLOBAL                   |
+----------------+----------------------------------------------------+


In [ ]:
!bs project download --id 347968211 --extension=fastq.gz -o /content/sample_data/sequence

In [8]:
%cd /content/sample_data/sequence

/content/sample_data/sequence


In [ ]:
#consolidate in one folder
!mkdir samples 
!find . -name "*.gz" -exec mv "{}" samples \;
!rmdir */

In [10]:
!mv \
/content/sample_data/hudson-valley-tick/v4-150bp-se-ref-seqs-gg.qza \
/content/sample_data/hudson-valley-tick/v4-150bp-classifier-gg.qza \
/content/sample_data/sequence

In [11]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path /content/sample_data/sequence/samples \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path demux-paired-end.qza

Imported /content/sample_data/sequence/samples as CasavaOneEightSingleLanePerSampleDirFmt to demux-paired-end.qza


In [12]:
!qiime tools peek demux-paired-end.qza

UUID:        8c441c42-c8fc-49a4-8e04-c44d450ef3b7
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [13]:
!qiime demux summarize \
  --i-data demux-paired-end.qza \
  --o-visualization demux-paired-end.qzv

Saved Visualization to: demux-paired-end.qzv


In [14]:
# This method denoises single-end sequences, dereplicates them, and filters chimeras.

!qiime dada2 denoise-single \
  --i-demultiplexed-seqs demux-paired-end.qza \
  --p-trim-left 0 \
  --p-trunc-len 150 \
  --o-representative-sequences se-rep-seqs-dada2.qza \
  --o-table se-table-dada2.qza \
  --o-denoising-stats se-stats-dada2.qza

Saved FeatureTable[Frequency] to: se-table-dada2.qza
Saved FeatureData[Sequence] to: se-rep-seqs-dada2.qza
Saved SampleData[DADA2Stats] to: se-stats-dada2.qza


In [ ]:
!qiime feature-table summarize \
  --i-table se-table-dada2.qza \
  --o-visualization se-rep-seqs-dada2.qzv \
  --m-sample-metadata-file /content/sample_data/Milli_Pooh02_220421_illumia_run[17346].txt

!qiime feature-table tabulate-seqs \
  --i-data se-rep-seqs-dada2.qza \
  --o-visualization se-rep-seqs-dada2.qzv

In [16]:
#open reference cluster using grene genes as referance
!qiime vsearch cluster-features-open-reference \
  --i-sequences se-rep-seqs-dada2.qza \
  --i-table se-table-dada2.qza \
  --i-reference-sequences v4-150bp-se-ref-seqs-gg.qza \
  --p-perc-identity .99 \
  --p-threads 0 \
  --o-clustered-table se-vs_clst99-table.qza \
  --o-clustered-sequences se-vs_clst99-seq.qza \
  --o-new-reference-sequences tick_refseq99_gg.qza

Saved FeatureTable[Frequency] to: se-vs_clst99-table.qza
Saved FeatureData[Sequence] to: se-vs_clst99-seq.qza
Saved FeatureData[Sequence] to: tick_refseq99_gg.qza


In [22]:
!qiime feature-table summarize \
  --i-table se-vs_clst99-table.qza \
  --o-visualization se-vs_clst99-table.qzv \
  --m-sample-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt

!qiime feature-table tabulate-seqs \
  --i-data se-vs_clst99-seq.qza \
  --o-visualization se-vs_clst99-seq.qzv

!qiime metadata tabulate \
  --m-input-file se-stats-dada2.qza \
  --o-visualization se-stats-dada2.qzv

Saved Visualization to: se-vs_clst99-table.qzv
Saved Visualization to: se-vs_clst99-seq.qzv
Saved Visualization to: se-stats-dada2.qzv


In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences se-vs_clst99-seq.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza

In [ ]:
!qiime feature-classifier classify-sklearn \
  --i-classifier v4-150bp-classifier-gg.qza \
  --i-reads se-vs_clst99-seq.qza \
  --o-classification taxonomy.qza

In [29]:
!qiime taxa barplot \
  --i-table se-vs_clst99-table.qza \
  --i-taxonomy taxonomy.qza \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --o-visualization taxa-bar-plots.qzv

Saved Visualization to: taxa-bar-plots.qzv


In [30]:
!qiime diversity core-metrics-phylogenetic \
  --i-phylogeny rooted-tree.qza \
  --i-table se-vs_clst99-table.qza \
  --p-sampling-depth 5800 \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --output-dir core-metrics-results3 

Saved FeatureTable[Frequency] to: core-metrics-results3/rarefied_table.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results3/faith_pd_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results3/observed_features_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results3/shannon_vector.qza
Saved SampleData[AlphaDiversity] to: core-metrics-results3/evenness_vector.qza
Saved DistanceMatrix to: core-metrics-results3/unweighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results3/weighted_unifrac_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results3/jaccard_distance_matrix.qza
Saved DistanceMatrix to: core-metrics-results3/bray_curtis_distance_matrix.qza
Saved PCoAResults to: core-metrics-results3/unweighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results3/weighted_unifrac_pcoa_results.qza
Saved PCoAResults to: core-metrics-results3/jaccard_pcoa_results.qza
Saved PCoAResults to: core-metrics-results3/bray_c

In [ ]:
!qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/sample_data/sequence/core-metrics-results2/faith_pd_vector.qza \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --o-visualization core-metrics-results/faith-pd-group-significance.qzv

!qiime diversity alpha-group-significance \
  --i-alpha-diversity /content/sample_data/sequence/core-metrics-results2/evenness_vector.qza \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --o-visualization core-metrics-results/evenness-group-significance.qzv

In [ ]:
!qiime diversity beta-group-significance \
  --i-distance-matrix core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --m-metadata-column category \
  --o-visualization core-metrics-results/unweighted-unifrac-group-significance.qzv \
  --p-pairwise

In [31]:
!conda install -c conda-forge deicode

Solving environment: \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [ ]:
!qiime deicode rpca \
  --i-table se-vs_clst99-table.qza \
  --p-min-feature-count 10 \
  --p-min-sample-count 500 \
  --o-biplot ordination.qza \
  --o-distance-matrix distance.qza

!qiime emperor biplot \
  --i-biplot ordination.qza \
  --m-sample-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --m-feature-metadata-file /content/sample_data/sequence/taxonomy.qza \
  --o-visualization biplot.qzv \
  --p-number-of-features 10

!qiime diversity beta-group-significance \
  --i-distance-matrix distance.qza \
  --m-metadata-file /content/sample_data/sequence/Milli_Pooh02_220421_illumia_run[17346].txt \
  --m-metadata-column category \
  --p-method permanova \
  --o-visualization status_significance.qzv